# Яндекс Музыка

Сравнение Москвы и Петербурга окружено мифами. Например:
 * Москва — мегаполис, подчинённый жёсткому ритму рабочей недели;
 * Петербург — культурная столица, со своими вкусами.

На данных Яндекс Музыки мы будем сравнивать поведение пользователей двух столиц.

**Цель исследования** — проверить три гипотезы:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. 
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

**Ход исследования**

Данные о поведении пользователей будут загружены из файла `yandex_music_project.csv`. О качестве данных ничего не известно, поэтому перед проверкой гипотез понадобится обзор данных, проверка на ошибки и поиск возможностей исправить самые критичные из них, если таковые будут иметься.
 
Таким образом, исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.

## 1 Обзор данных

### 1.1 Загрузим pandas, импортируем и проведем первичный осмотр данных

Импортируем `pandas`:

In [1]:
import pandas as pd

Читаем файл `yandex_music_project.csv` из папки `/datasets`, сохраним его в переменной `df`:

In [2]:
df = pd.read_csv('/datasets/yandex_music_project.csv')

Посмотрим первые десять строк таблицы:

In [3]:
print(df.head(10))

     userID                        Track            artist   genre  \
0  FFB692EC            Kamigata To Boots  The Mass Missile    rock   
1  55204538  Delayed Because of Accident  Andreas Rönnberg    rock   
2    20EC38            Funiculì funiculà       Mario Lanza     pop   
3  A3DD03C9        Dragons in the Sunset        Fire + Ice    folk   
4  E2DC1FAE                  Soul People        Space Echo   dance   
5  842029A1                    Преданная         IMPERVTOR  rusrap   
6  4CB90AA5                         True      Roman Messer   dance   
7  F03E1C1F             Feeling This Way   Polina Griffith   dance   
8  8FA1D3BE     И вновь продолжается бой               NaN  ruspop   
9  E772D5C0                    Pessimist               NaN   dance   

             City        time        Day  
0  Saint-Petersburg  20:28:33  Wednesday  
1            Moscow  14:07:09     Friday  
2  Saint-Petersburg  20:58:07  Wednesday  
3  Saint-Petersburg  08:37:09     Monday  
4            M

С помощью `info()` получим общую информацию о таблице:

In [4]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB
None


Итак, в таблице семь столбцов. Тип данных во всех столбцах — `object`.

Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. 

Предварительно можно утверждать, что данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

Нужно устранить проблемы в данных.

## 2 Предобработка данных
Исправим стиль в заголовках столбцов, исключим пропуски. Затем проверим данные на дубликаты.

### 2.1 Стиль заголовков
Выведем на экран названия столбцов:

In [6]:
# перечень названий столбцов таблицы df
print(df.columns)

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')


Приведём названия в соответствие с хорошим стилем:
* несколько слов в названии запишем в «змеином_регистре»,
* все символы сделаем строчными,
* устраним пробелы.

Переименуем колонки так:
* `'  userID'` → `'user_id'`;
* `'Track'` → `'track'`;
* `'  City  '` → `'city'`;
* `'Day'` → `'day'`.

In [7]:
# переименование столбцов
df = df.rename(columns={'  userID': 'user_id', 'Track': 'track', '  City  ': 'city', 'Day': 'day'})

Проверим получившийся результат:

In [1]:
print(df.columns)

NameError: name 'df' is not defined

### 2.2 Обработаем пропуски

Посчитаем, сколько в таблице пропущенных значений:

In [9]:
# подсчёт пропусков
print(df.isna().sum())

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64


Не все пропущенные значения влияют на исследование. Так в `track` и `artist` пропуски не важны для целей работы. Достаточно заменить их явными обозначениями.

Но пропуски в `genre` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге. На практике было бы правильно установить причину пропусков и восстановить данные. Но такой возможности на данный момент. Придётся:
* заполнить и эти пропуски явными обозначениями;
* оценить, насколько они повредят расчётам. 

####  2.2.1 Заменим пропущенные значения в столбцах `track`, `artist` и `genre` на строку `'unknown'`

Для этого создадим список `columns_to_replace`, переберём его элементы циклом `for` и для каждого столбца выполним замену пропущенных значений:

In [10]:
# перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'
columns_to_replace = []

for value in df:
        df['track'] = df['track'].fillna('unknown')
        df['artist'] = df['artist'].fillna('unknown')
        df['genre'] = df['genre'].fillna('unknown')

Проверим, есть ли пропуски сейчас:

In [11]:
# подсчёт пропусков
print(df.isna().sum())

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64


### 2.3 Обработка дубликатов

#### 2.3.1 Обработка явных дубликатов

Посчитаем явные дубликаты в таблице:

In [12]:
# подсчёт явных дубликатов
print(df.duplicated().sum())

3826


Вызовем drop_duplicates, чтобы их удалить:

In [13]:
# удаление явных дубликатов
df = df.drop_duplicates()

Убедимся, что полностью избавились от дублей:

In [14]:
# проверка на отсутствие дубликатов
print(df.duplicated().sum())

0


#### 2.3.2 Обработка неявных дубликатов

Теперь избавимся от неявных дубликатов в колонке `genre`. Например, название одного и того же жанра может быть записано немного по-разному. Такие ошибки тоже повлияют на результат исследования.

Для этого выведем на экран список уникальных названий жанров, отсортированный в алфавитном порядке:
1. извлечём нужный столбец датафрейма; 
2. применим к нему метод сортировки;
3. для отсортированного столбца вызовем метод, который вернёт уникальные значения из столбца.

In [15]:
# Просмотр уникальных названий жанров
genre_sorted_names = df['genre'].sort_values()
print(genre_sorted_names.unique())

['acid' 'acoustic' 'action' 'adult' 'africa' 'afrikaans' 'alternative'
 'alternativepunk' 'ambient' 'americana' 'animated' 'anime' 'arabesk'
 'arabic' 'arena' 'argentinetango' 'art' 'audiobook' 'author' 'avantgarde'
 'axé' 'baile' 'balkan' 'beats' 'bigroom' 'black' 'bluegrass' 'blues'
 'bollywood' 'bossa' 'brazilian' 'breakbeat' 'breaks' 'broadway'
 'cantautori' 'cantopop' 'canzone' 'caribbean' 'caucasian' 'celtic'
 'chamber' 'chanson' 'children' 'chill' 'chinese' 'choral' 'christian'
 'christmas' 'classical' 'classicmetal' 'club' 'colombian' 'comedy'
 'conjazz' 'contemporary' 'country' 'cuban' 'dance' 'dancehall' 'dancepop'
 'dark' 'death' 'deep' 'deutschrock' 'deutschspr' 'dirty' 'disco' 'dnb'
 'documentary' 'downbeat' 'downtempo' 'drum' 'dub' 'dubstep' 'eastern'
 'easy' 'electronic' 'electropop' 'emo' 'entehno' 'epicmetal' 'estrada'
 'ethnic' 'eurofolk' 'european' 'experimental' 'extrememetal' 'fado'
 'fairytail' 'film' 'fitness' 'flamenco' 'folk' 'folklore' 'folkmetal'
 'folkrock' 

Есть следующие неявные дубликаты:
* *hip*,
* *hop*,
* *hip-hop*.

Чтобы очистить от них таблицу используем метод `replace()` с двумя аргументами: списком строк-дубликатов (включащий *hip*, *hop* и *hip-hop*) и строкой с правильным значением - `hiphop`:

In [16]:
# Устранение неявных дубликатов
duplicates = ['hip', 'hop', 'hip-hop'] # создаём список дупликатов, чтобы не плодить метод replace()
df['genre'] = df['genre'].replace(duplicates, 'hiphop') # смена значений дупликатов на значение right_name

Убедимся, что заменили неправильные названия, снова выведя отсортированный список уникальных значений столбца `genre`:

In [17]:
genre_sorted_names = df['genre'].sort_values()
print(genre_sorted_names.unique())

['acid' 'acoustic' 'action' 'adult' 'africa' 'afrikaans' 'alternative'
 'alternativepunk' 'ambient' 'americana' 'animated' 'anime' 'arabesk'
 'arabic' 'arena' 'argentinetango' 'art' 'audiobook' 'author' 'avantgarde'
 'axé' 'baile' 'balkan' 'beats' 'bigroom' 'black' 'bluegrass' 'blues'
 'bollywood' 'bossa' 'brazilian' 'breakbeat' 'breaks' 'broadway'
 'cantautori' 'cantopop' 'canzone' 'caribbean' 'caucasian' 'celtic'
 'chamber' 'chanson' 'children' 'chill' 'chinese' 'choral' 'christian'
 'christmas' 'classical' 'classicmetal' 'club' 'colombian' 'comedy'
 'conjazz' 'contemporary' 'country' 'cuban' 'dance' 'dancehall' 'dancepop'
 'dark' 'death' 'deep' 'deutschrock' 'deutschspr' 'dirty' 'disco' 'dnb'
 'documentary' 'downbeat' 'downtempo' 'drum' 'dub' 'dubstep' 'eastern'
 'easy' 'electronic' 'electropop' 'emo' 'entehno' 'epicmetal' 'estrada'
 'ethnic' 'eurofolk' 'european' 'experimental' 'extrememetal' 'fado'
 'fairytail' 'film' 'fitness' 'flamenco' 'folk' 'folklore' 'folkmetal'
 'folkrock' 

**Выводы**

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

Мы исправили заголовки, чтобы упростить работу с таблицей, а без дубликатов исследование станет более точным.

Пропущенные значения заменены на `'unknown'`. Ещё предстоит увидеть, не повредят ли исследованию пропуски в колонке `genre`.

Теперь можно перейти к проверке гипотез. 

## 3 Проверка гипотез

### 3.1 Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделим пользователей Москвы и Санкт-Петербурга.
* Сравним, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.

Сначала оценим активность пользователей в каждом городе. Сгруппируем данные по городу и посчитаем прослушивания в каждой группе.

In [18]:
# Подсчёт прослушиваний в каждом городе
city_grouping = df.groupby('city')['time'].count()
print(city_grouping)

city
Moscow              42741
Saint-Petersburg    18512
Name: time, dtype: int64


В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.


Теперь сгруппируем данные по дню недели и посчитаем прослушивания в понедельник, среду и пятницу (в данных есть информация о прослушиваниях только за эти дни):

In [19]:
# Подсчёт прослушиваний в каждый из трёх дней
day_grouping = df.groupby('day')['city'].count()
print(day_grouping)

day
Friday       21840
Monday       21354
Wednesday    18059
Name: city, dtype: int64


В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

Напишем функцию `number_tracks()`, которая объединит два эти расчёта.

`number_tracks()` посчитает прослушивания для заданного дня и города. Ей понадобятся два параметра:
* день недели,
* название города.

В функции мы сохраним в переменную строки исходной таблицы, у которых значение:
  * в колонке `day` равно параметру `day`,
  * в колонке `city` равно параметру `city`.

Для этого применим последовательную фильтрацию с логической индексацией. С её помощью `number_tracks()` сначала получит из исходной таблицы строки с нужным днём, затем из результата отфильтрует строки с нужным городом, методом count() посчитает количество значений в колонке `user_id` и вернет результат:

In [2]:
# функция с двумя параметрами: day, city.
def number_tracks(day, city):    
# В переменной track_list сохраняются те строки таблицы df, для которых 
# значение в столбце 'day' равно параметру day и одновременно значение
# в столбце 'city' равно параметру city (последовательная фильтрация с логической индексацией).
    track_list = df[df['day'] == day]
    track_list = track_list[track_list['city'] == city]
# В переменной track_list_count сохраняется число значений столбца 'user_id',
# рассчитанное методом count() для таблицы track_list.
    track_list_count = track_list['user_id'].count()
# Функция возвращает число - значение track_list_count.
    return track_list_count

Теперь вызовем `number_tracks()` шесть раз, меняя значение параметров — так, чтобы получить данные для каждого города в каждый из трёх дней.

In [289]:
# количество прослушиваний в Москве по понедельникам
print(number_tracks('Monday', 'Moscow'))

15740

In [290]:
# количество прослушиваний в Санкт-Петербурге по понедельникам
print(number_tracks('Monday', 'Saint-Petersburg'))

5614

In [291]:
# количество прослушиваний в Москве по средам
print(number_tracks('Wednesday', 'Moscow'))

11056

In [292]:
# количество прослушиваний в Санкт-Петербурге по средам
print(number_tracks('Wednesday', 'Saint-Petersburg'))

7003

In [293]:
# количество прослушиваний в Москве по пятницам
print(number_tracks('Friday', 'Moscow'))

15945

In [294]:
# количество прослушиваний в Санкт-Петербурге по пятницам
print(number_tracks('Friday', 'Saint-Petersburg'))

5895

Создадим новую таблицу и занесём в неё получившиеся значения, где:

* названия колонок — `['city', 'monday', 'wednesday', 'friday']`;
* данные — результаты, полученные с помощью `number_tracks`.

In [296]:
# Список данных для таблицы:
amount_play = [
    ['Moscow', 15740, 11056, 15945],
    ['Saint-Petersburg', 5614, 7003, 5895],
]
# Список столбцов:
cities_by_days = ['city', 'monday', 'wednesday', 'friday']

# Таблица с результатами
cities_music = pd.DataFrame(data=amount_play, columns=cities_by_days)

print(cities_music)

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### 3.2 Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Сохраним таблицы с данными в две переменные:
* по Москве — в `moscow_general`;
* по Санкт-Петербургу — в `spb_general`.

In [297]:
# получение таблицы moscow_general из тех строк таблицы df, 
# для которых значение в столбце 'city' равно 'Moscow'
moscow_general = df[df['city'] == 'Moscow']

In [298]:
# получение таблицы spb_general из тех строк таблицы df,
# для которых значение в столбце 'city' равно 'Saint-Petersburg'
spb_general = df[df['city'] == 'Saint-Petersburg']

Создадим функцию `genre_weekday()` с четырьмя параметрами:
* таблица (датафрейм) с данными,
* день недели,
* начальная временная метка в формате 'hh:mm', 
* последняя временная метка в формате 'hh:mm'.

Функция должна вернуть информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [300]:
# Объявление функции genre_weekday() с параметрами датафрейма df, day, time1, time2:
def genre_weekday(df, day, time1, time2):
# 1) в переменную genre_df сохраняются те строки переданного датафрейма df, для
#    которых одновременно:
#    - значение в столбце day равно значению аргумента day
#    - значение в столбце time больше значения аргумента time1
#    - значение в столбце time меньше значения аргумента time2
#    (последовательная фильтрация с помощью логической индексации)
    genre_df = df[df['day'] == day]
    genre_df = genre_df[genre_df['time'] > time1]
    genre_df = genre_df[genre_df['time'] < time2]
# 2) группируем датафрейм genre_df по столбцу genre, берем один из его
#    столбцов и считаем count() количество записей для каждого из
#    присутствующих жанров, получившийся Series записываем в переменную
#    genre_df_count
    genre_df_count = genre_df.groupby('genre')['genre'].count()
# 3) сортируем genre_df_count по убыванию встречаемости и сохраняем
#    в переменную genre_df_sorted
    genre_df_sorted = genre_df_count.sort_values(ascending=False)
# 4) вернем Series из 10 первых значений genre_df_sorted, это будут топ-10
#    популярных жанров (в указанный день, в заданное время)    
    return genre_df_sorted[:10]

Cравним результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [301]:
# вызов функции для утра понедельника в Москве (вместо df — таблица moscow_general):
print(genre_weekday(moscow_general, 'Monday','07:00', '11:00'))

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [302]:
# вызов функции для утра понедельника в Петербурге (вместо df — таблица spb_general):
print(genre_weekday(spb_general, 'Monday', '07:00', '11:00'))

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [303]:
# вызов функции для вечера пятницы в Москве:
print(genre_weekday(moscow_general, 'Friday', '17:00', '23:00'))

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [304]:
# вызов функции для вечера пятницы в Петербурге:
print(genre_weekday(spb_general, 'Friday', '17:00', '23:00'))

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### 3.3 Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

Чтобы проверить эту гипотезу, для начала сгруппируем таблицу `moscow_general` по жанру и посчитаем прослушивания треков каждого жанра методом `count()`. Затем отсортируем результат в порядке убывания и сохраним его в таблице `moscow_genres`.

In [306]:
moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending=False)

Выведем на экран первые десять строк `moscow_genres`:

In [305]:
# просмотр первых 10 строк moscow_genres
print(moscow_genres.head(10))

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

Теперь повторим то же и для Петербурга:

In [307]:
spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending=False)

Выведем на экран первые десять строк `spb_genres`:

In [308]:
# просмотр первых 10 строк spb_genres
print(spb_genres.head(10))

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


## Итоги исследования

Были проверены три гипотезы и установлено следующее:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.